<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex06notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex06notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 演習課題
---


In [ ]:
# いろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 国数英データの主成分分析

「国語」「数学」「英語」3科目の点数を集めたデータに主成分分析を適用してみましょう．


#### データを入手し，前処理してから観察する

次のコードセルを実行すると，データを入手します．

In [ ]:
# 国数英
dfJME = pd.read_table('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/jme.txt', sep=' ', names=['国語', '数学', '英語'])
dfJME

見てのとおり，3科目の点数を30人分集めたデータです．
後の計算のためにこれを NumPy array に変換し，3科目それぞれの平均が 0 になるようにしておきます．

In [ ]:
# NumPy array にする
Xorg = dfJME.to_numpy()
print('Xorg の最初の5行')
print(Xorg[:5, :], Xorg.shape)
print()
# 平均を差し引いたものを X とする
Xm = np.mean(Xorg, axis=0)
X = Xorg - Xm
print('X の最初の5行')
print(X[:5, :], X.shape)

3科目中の2科目のペア3通りすべてについて散布図を描くと，次のようになっています．

In [ ]:
# X の散布図
fig, ax = plt.subplots(1, 3, figsize=(10, 4))

vname = ['Japanese', 'Math', 'English']
index = [(0, 1), (0, 2), (1, 2)]

for n in range(len(index)):
    i, j = index[n]
    ax[n].scatter(X[:, i], X[:, j])
    ax[n].set_xlim(-30, 30)
    ax[n].set_ylim(-30, 30)
    ax[n].set_aspect('equal')
    ax[n].axhline(0, color='gray')
    ax[n].axvline(0, color='gray')
    ax[n].set_xlabel(vname[i])
    ax[n].set_ylabel(vname[j])

fig.tight_layout()
plt.show()

#### 分散共分散行列とその固有値固有ベクトルを求める

主成分分析ではデータの分散共分散行列を求め，その固有値・固有ベクトルを求める必要があります．
ここでは，その計算のコードは Python + NumPy で書いています．

まずは分散共分散行列を求めます．

In [ ]:
# X はゼロ平均なので以下で V が得られる
V = X.T @ X / X.shape[0]
print('V = ')
print(V, V.shape)

`V` の要素を観察すると，次のようなことが分かります．ここでは，`V` の `i` 行目 `j` 列目の要素を `V[i, j]` と書いて説明してます（`i` や `j` は 0 から数えてます）．

- 科目毎に見ると，「数学」の分散（`V[1, 1]`）が大きく，「英語」の分散（`V[2, 2]`）が小さい
- どの2科目間の共分散も正である．このことは，散布図からも読みとれますね．

分散共分散行列の固有値と固有ベクトルを求めます（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注： 【この説明は，線形代数関係の数値計算に興味のある人向けの内容です】ここでは，<a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html">np.linalg.svd</a> という関数を使って固有値固有ベクトルを求めています．この関数は本来，「特異値分解(Singular Value Decomposition, SVD)」のためのものですが，引数に実対象行列を渡した場合，その行列の固有値固有ベクトルを返してくれます．この他，対称行列（正確にはエルミート行列）の固有値固有ベクトルを求める <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html">np.linalg.eigh</a> や一般の正方行列の固有値固有ベクトルを求める <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html">np.linalg.eig</a> なども使えますが，np.linalg.svd が最も便利で実用的です．
</span>

In [ ]:
# V の固有値固有ベクトルを求める
U, eval, Vt = np.linalg.svd(V)  # eval が固有値，U が固有ベクトルをならべた行列になる
print('U = ')
print(U, U.shape)
print('eval = ')
print(eval)
print()
for d in range(len(eval)): # 固有ベクトルは列ベクトルとして U に格納されてる
    print(f'固有値 {eval[d]:.2f} に対応する固有ベクトルは {U[:, d]}')
print()
print('U.T @ U = ')
print(U.T @ U)

3次元のデータなので，分散共分散行列は $3\times 3$ です．したがって，固有値は 3 つ，固有ベクトルは3つの固有値のそれぞれに対応して一つずつあるので計 3 つ得られます．それぞれの固有ベクトルは 3 次元ベクトルです．

`U.T @ U` のところは，`U` が直交行列になっていることを確認しています．この結果から，3つの固有ベクトルは，いずれもノルムが 1 であり，互いに直交していることが分かります．

#### 次元削減の変換を行い結果を考察する

3つの固有ベクトルのうち固有値の大きい方二つをならべて，主成分分析による次元削減のための変換行列（notebookA の $U$，↓のコードセルでの変数名は `U2`）を作り，これを使ってデータを変換します．

In [ ]:
# 固有値の大きい方2つに対応する固有ベクトルをならべた行列を U2 とする
U2 = U[:, :2]
print('U2 = ')
print(U2, U2.shape)
print()
# U2 を使って X を Y に変換
Y = X @ U2
print('Y = X @ U2 の最初の5行')
print(Y[:5, :], Y.shape)

次元削減の変換によって，3次元のデータを2次元にすることができました．
その散布図を描いてみると，このようになります．

In [ ]:
# Y の散布図
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(Y[:, 0], Y[:, 1])
idx = [6, 14, 29]
ax.scatter(Y[idx, 0], Y[idx, 1])
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_aspect('equal')
ax.axhline(0, color='gray')
ax.axvline(0, color='gray')
ax.set_xlabel(r'$y_1$')
ax.set_ylabel(r'$y_2$')
for i in idx:
    plt.annotate(f'{i}', (Y[i, 0]+2, Y[i, 1]))
plt.show()

ここで，3科目の点数と，それを2次元にした値（2つの主成分）との関係を理解するために，
$\pmb{y} = U^{\top}\pmb{x}$ の式を具体的な固有ベクトルの値を使って表してみると，次のようになります．

$$
\pmb{y} =
\begin{pmatrix}
y_1 \\ y_2
\end{pmatrix}
=
U^{\top}\pmb{x}
=
\begin{pmatrix}
\pmb{u}_1\cdot\pmb{x} \\ \pmb{u}_2\cdot\pmb{x}
\end{pmatrix}
=
\begin{pmatrix}
-0.540 x_1 - 0.603 x_2 - 0.588 x_3\\
0.736 x_1 - 0.677 x_2 + 0.0174 x_3
\end{pmatrix}
$$

ここで，$x_1, x_2, x_3$ は元のデータからそれぞれの平均を引いて得られた値ですので，元の点数がその科目の平均より低ければ負に，高ければ正になります．
一方，第1主成分 $y_1$ の式を見ると，$x_1, x_2, x_3$ の係数はいずれも負で，値の大きさがほぼ同じです．このことから，第1主成分 $y_1$ は「3科目の総合力」（ただし値が小さい方が総合力が高い）を表すと解釈できます．

一方，第2主成分 $y_2$ の方は，英語の係数が 0 に近く影響が小さいので無視できると考えると，

「国語が低く数学が高い」 $y_2 \mbox{が負} \longleftrightarrow y_2 が正$「国語が高く数学が低い」

という関係があるとみなせます．したがって，$y_2$ は「文理度」のようなものと解釈できます（値が小さいと理系っぽい，大きいと文系っぽい）（注）．

これらのことから，ここで行った主成分分析による次元削減では，国語，数学，英語3科目の点数のデータを2つの成分で（2つの変数で）説明しようとしたら，「3科目の総合力」と「文理度」という2つが得られた，この2つがこのデータの主な成分である，という解釈をすることができます．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注： 説明のわかりやすさを優先して，国語が得意で数学が苦手なら「文系」，逆なら「理系」という乱暴な解釈をしています．
</span>

上記のことを，具体的な数値で確認してみましょう．次のコードセルを実行すると，散布図に記を付けた3人のデータを表示します．

In [ ]:
print('Xorg の平均: ', Xm)
print()
for i in idx:
    print(f'No.{i:2d}  Xorg: {Xorg[i, :]}  X: [{X[i, 0]:.1f} {X[i, 1]:.1f} {X[i, 2]:.1f}]  Y: [{Y[i, 0]:.1f} {Y[i, 1]:.1f}]')

これら3人の主成分スコアの値を見て，「総合力」，「文理度」を考察すると，次のような結果が得られます．次のコードセルを実行すると解答例が表示されます．まずは自分で考えてから実行してね．


In [ ]:
Q = b'Ci0gTm8uNiDjga/vvIznrKwx5Li75oiQ5YiG44K544Kz44Ki44GM5aSn44GN44GE44Gu44GnM+enkeebruOBrue3j+WQiOWKm+OBr+S9juOBj++8jOesrDLkuLvmiJDliIbjgrnjgrPjgqLjga/mraPjgaDjgYww44Gr6L+R44GE44Gu44Gn44KE44KE5paH57O75a+E44KK44Go44GE44GI44KL77yOCi0gTm8uMTQg44Gv77yM56ysMeS4u+aIkOWIhuOCueOCs+OCouOBryBOby42IOOBqOOBu+OBvOWQjOOBmO+8jOesrDLkuLvmiJDliIbjgrnjgrPjgqLjga/osqDjgarjga7jgafnkIbns7vlr4TjgorvvI4KLSBOby4yOSDjga/vvIznrKwx5Li75oiQ5YiG44K544Kz44Ki44GM5bCP44GV44GE44Gu44GnM+enkeebruOBrue3j+WQiOWKm+OBjOmrmOOBj++8jOesrDLkuLvmiJDliIbjgrnjgrPjgqLjga/mraPjgarjga7jgafmlofns7vlr4TjgorvvI4K'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

このデータの場合は，元データもたかだか3次元なので，それぞれの元の点数と平均点からもそのような傾向を確認できますね．

#### 問題1

ここまでの内容を理解しなさい

---
### 外食支出金額データの主成分分析

#### データを入手して前処理する

「SSDSE-家計消費」というデータセットを読み込んで前処理を行うコードを実行します．
データについての説明も表示します．

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/DiningOutSpendingData.py
from DiningOutSpendingData import DiningOutSpendingData
dosd = DiningOutSpendingData()
kinki5 = dosd.kinki5()
tohoku6 = dosd.tohoku6()
dosd.info()

次のコードセルを実行すると，読み込んだデータに前処理を適用して，最初の5件のデータを表示します．notebookA では 13 個の変数を全て使っていましたが，ここではいくつか削除してから分析を行うことにします．

In [ ]:
# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1
# DataFrame を入手して最初の5件を表示
dfFeature_raw = dosd.getDataFrame()
# いくつかの列を削除
dfFeature = dfFeature_raw.drop(columns=['他の麺類外食', '他の主食的外食', '学校給食', '飲酒代'])
dfFeature.head(5)

In [ ]:
# NumPy array を作る
Xorg = dfFeature.drop(columns=['SSDSE-C-2022', 'City']).to_numpy()
print('Xorg の最初の5行')
print(Xorg[:5, :], Xorg.shape)
print()

#### 主成分分析によって2次元に次元削減する

次のセルを実行すると，上記のデータに主成分分析を適用して2次元に変換したものが得られます（注）．
ここでは，[statsmodels](https://www.statsmodels.org/) という Python 向け統計分析ライブラリを使っています（参考: [statsmodels.multivariate.pca.PCA](https://www.statsmodels.org/dev/generated/statsmodels.multivariate.pca.PCA.html) ）

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注： ここでは，元のデータの変数値（支出金額）そのままではなく，それらを変数ごとに標準化したデータに対して主成分分析を適用しています（<tt>PCA()</tt> の <tt>standardize</tt> オプションを <tt>True</tt> にしている）．
</span>

In [ ]:
# 主成分分析を利用してデータの次元数を2次元に削減（標準化あり）
from statsmodels.multivariate.pca import PCA
pca = PCA(Xorg, ncomp=2, standardize=True, demean=True, normalize=False)
Y = pca.scores
print('Y の最初の5行')
print(Y[:5, :], Y.shape)

`Y` の `shape` は `(47, 2)` です．47都市の特徴を2つの数値で表したものとなっています．

2次元のデータなので，散布図に描けます．描いてみるとこんなん：

In [ ]:
# Y の散布図
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[kinki5, 0], Y[kinki5, 1], label='kinki5')
ax.scatter(Y[tohoku6, 0], Y[tohoku6, 1], label='tohoku6')
ax.set_xlim(-7.5, 7.5)
ax.set_ylim(-7.5, 7.5)
ax.set_aspect('equal')
ax.axhline(0, color='gray') # x軸
ax.axvline(0, color='gray') # y軸
ax.legend()
for n in range(len(Y)):
    plt.annotate(f'{n}', (Y[n, 0]+0.1, Y[n, 1]+0))
plt.show()

In [ ]:
# 変換行列の値を得る
Ut = pca.coeff # Ut[h, :] が分散共分散行列の h 番目に大きな固有値に対応する固有ベクトル
print(Ut, Ut.shape)

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 2
dfU = pd.DataFrame(Ut, index=['u1', 'u2'], columns=dfFeature.drop(columns=['SSDSE-C-2022', 'City']).columns)
dfU

`u1` の行は第1主成分に対応する変換ベクトル（分散共分散行列の最大固有値に対する固有ベクトル）の値を表します．ここから，次のようなことが読み取れます．

> `u1` の値は全て正なので，全ての変数 $x_{d}$（$d = 1, 2, \ldots, 13$）の値が正ならば（元の値が平均より大きければ），第1主成分スコアは正になる．全て負ならば（元の値が平均より小さければ），負になる．
大まかに言えば，第1主成分スコアは「分析対象の外食支出の総額の多さ」に対応している．

第2主成分スコアについても，`u2` を見て同様の考察をすることができる．



#### 問題2

次のことをやりなさい：

1. ここまでの内容を理解する
1. `u2` の値を見て，次のものをメモしておく：
    - 係数の値が大きい変数の名前
    - 係数の値が負で絶対値が大きい変数の名前
1. 次のことを考える：
    - 散布図で 20, 46 とラベル付けされた2つの都市のうち，分析対象となっている外食支出項目の総額が多い傾向にあるのはどちらか
    - 散布図で 5, 27 とラベル付けされた2つの都市の外食支出の傾向は，どのように異なるか
    - `kinki5`, `tohoku6` とラベル付けされた都市グループそれぞれの外食支出はどのような傾向にあるか